Effets des variables macroéconomiques sur le CAC 40

In [33]:
import pandas as pd
import numpy as np
from datetime import datetime
!pip install openpyxl

Chargement des fichiers Excel qui forment la base de donnée

In [34]:
chomage = pd.read_excel('Chômage_depuis1975.xlsx')
confiance = pd.read_excel('Confiance_ménages.xlsx')
ev_pib = pd.read_excel('Evolution_PIB_trimestre.xlsx')
ipc = pd.read_excel('IPC_depuis1991.xlsx')

Nettoyage des données

In [35]:
chomage.head()

,Tableau 1. Taux de chômage trimestriel au sens du BIT en France (hors Mayotte) par sexe et âge,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,"Données CVS en moyenne trimestrielle, en %",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,En jaune : Les séries ont été constituées à pa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,HOMMES (%),NaN,NaN,NaN,FEMMES (%),NaN,NaN,NaN,TOTAL (%),NaN,NaN,NaN,Nombre de chômeurs
4,trimestre.année,Âge,NaN,NaN,Total,Âge,NaN,NaN,Total,Âge,NaN,NaN,Total,NaN


In [36]:
# Renommer les colonnes à utiliser
chomage.rename(columns={'Unnamed: 12': 'taux_chomage'}, inplace=True)
chomage.rename(columns={'Tableau 1. Taux de chômage trimestriel au sens du BIT en France (hors Mayotte) par sexe et âge': 'trimestre'}, inplace=True) 

In [37]:
print(chomage.columns)

Index(['trimestre', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'taux_chomage', 'Unnamed: 13'],
      dtype='object')


In [38]:
# Supprimer les lignes ou colonnes contenant des valeurs inutiles (nan, descriptions, etc.)
chomage = chomage.dropna(axis=1, how='all')  # Supprime les colonnes complètement vides
chomage = chomage.dropna(axis=0, how='all')  # Supprime les lignes complètement vides

In [39]:
# Supprimer les colonnes inutiles pour notre analyse
colonnes_a_supprimer = [
    'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 
    'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 
    'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 13'
]

# Supprimer les colonnes
chomage.drop(columns=colonnes_a_supprimer, inplace=True)

# Afficher les premières lignes du après suppression
chomage.head()

,trimestre,taux_chomage
0,"Données CVS en moyenne trimestrielle, en %",NaN
2,En jaune : Les séries ont été constituées à pa...,NaN
3,NaN,NaN
4,trimestre.année,Total
5,NaN,NaN


In [40]:
# Trouver l'index de la valeur 'T1.2000' dans la colonne 'trimestre'
index_t1_2000 = chomage[chomage['trimestre'] == 'T1.2000'].index

# Afficher l'index
print(index_t1_2000)


Index([106], dtype='int64')


In [41]:
import pandas as pd

# On supprime les premières lignes inutiles pour que les données débutent au premier trimestre  
chomage_clean = chomage.iloc[105:].reset_index(drop=True)

# Afficher les premieres lignes des données nettoyées
chomage_clean.head()


,trimestre,taux_chomage
0,T1.2000,9.1
1,T2.2000,8.7
2,T3.2000,8.4
3,T4.2000,8
4,T1.2001,7.8


Nettoyage des données sur la confiance 

In [42]:
confiance.head()

,Opinion des ménages-Monthly confidence consumer survey,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,Données corrigées des variations saisonnières-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DATE,Indicateur synthétique,Niveau de vie en France - évolution passée,Niveau de vie en France - perspectives d'évolu...,Chômage - perspectives d'évolution,Prix - évolution passée,Prix - perspectives d'évolution,Opportunité de faire des achats importants,Opportunité d'épargner,Capacité d'épargne actuelle,Situation financière personnelle - évolution p...,Situation financière personnelle - perspective...,Capacité d'épargne future
3,NaN,Synthetic index,"General economic situation, past 12 months","General economic situation, next 12 months","Unemployment, next 12 months","Consumer prices, past 12 months","Consumer prices, next 12 months","Major purchases intentions, next 12 months","Savings intentions, next 12 months",Current saving capacity,"Financial situation, past 12 months","Financial situation, next 12 months",Expected saving capacity
4,moyenne/average,100,-47.749835,-27.733798,32.585969,-13.383858,-31.095762,-15.188091,18.029779,9.656539,-20.980754,-6.560103,-7.110802


In [43]:
# Renommer les colonnes à utiliser
confiance.rename(columns={'Unnamed: 1': 'indic_confiance'}, inplace=True)
confiance.rename(columns={'Opinion des ménages-Monthly confidence consumer survey': 'date'}, inplace=True) 

In [44]:
print(confiance.columns)

Index(['date', 'indic_confiance', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12'],
      dtype='object')


In [45]:
# Supprimer les colonnes inutiles pour notre analyse
colonnes_a_supprimer = [
    'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12'
]

# Supprimer les colonnes
confiance.drop(columns=colonnes_a_supprimer, inplace=True)

# Afficher les premières lignes du après suppression
confiance.head()

,date,indic_confiance
0,Données corrigées des variations saisonnières-...,NaN
1,NaN,NaN
2,DATE,Indicateur synthétique
3,NaN,Synthetic index
4,moyenne/average,100


In [46]:
# S'assurer que la colonne 'date' est au format datetime
confiance['date'] = pd.to_datetime(confiance['date'], errors='coerce')

# Filtrer les données à partir de janvier 2000
confiance_clean =confiance[confiance['date'] >= '2000-01-01']

# Afficher les premières lignes des données filtrées
confiance_clean.head()

/tmp/ipykernel_21814/606775895.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  confiance['date'] = pd.to_datetime(confiance['date'], errors='coerce')


,date,indic_confiance
204,2000-01-01,119.186742
205,2000-02-01,119.010085
206,2000-03-01,119.930046
207,2000-04-01,122.586881
208,2000-05-01,122.625812


Nettoyage de la table Evolution du PIB

On cherche à isoler des indicateurs pour l'évolution du PIB, la consommation des ménages et des APU, la formation brute de capital fixe, les importations et exportations. On fait le choix de prendre plus d'indicateurs, quitte à les éliminer par la suite.

In [47]:
ev_pib.head()

,Évolution du produit intérieur brut et de ses composantes,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,par rapport au trimestre précédent en volume en %,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Trimestre,Produit intérieur brut (PIB),Importations,Dépense de consommation des ménages,Dépense de consommation des APU1,Formation brute de capital fixe,dont :,NaN,NaN,Exportations,Contributions :,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,Entreprises non financières,ménages,APU1,NaN,Demande intérieure finale hors stocks,Variations de stocks,Commerce extérieur
4,2024-T3,0.362294,-0.684306,0.515766,0.513113,-0.84265,-1.365417,-0.887841,0.472325,-0.470732,0.212544,0.069318,0.080432


In [48]:
# Renommer les colonnes à utiliser
ev_pib.rename(columns={'Évolution du produit intérieur brut et de ses composantes': 'trimestre'}, inplace=True)
ev_pib.rename(columns={'Unnamed: 1': 'pib'}, inplace=True) 
ev_pib.rename(columns={'Unnamed: 2': 'importations'}, inplace=True) 
ev_pib.rename(columns={'Unnamed: 3': 'conso_menages'}, inplace=True) 
ev_pib.rename(columns={'Unnamed: 4': 'conso_apu'}, inplace=True) 
ev_pib.rename(columns={'Unnamed: 5': 'fbcf'}, inplace=True) 
ev_pib.rename(columns={'Unnamed: 9': 'exportations'}, inplace=True) 

In [49]:
print(ev_pib.columns)

Index(['trimestre', 'pib', 'importations', 'conso_menages', 'conso_apu',
       'fbcf', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'exportations',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12'],
      dtype='object')


In [50]:
# Supprimer les colonnes inutiles pour notre analyse
colonnes_a_supprimer = [
    'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8',
    'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12']

# Supprimer les colonnes
ev_pib.drop(columns=colonnes_a_supprimer, inplace=True)

# Afficher les premières lignes du après suppression
ev_pib.head()

,trimestre,pib,importations,conso_menages,conso_apu,fbcf,exportations
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,par rapport au trimestre précédent en volume en %,NaN,NaN,NaN,NaN,NaN,NaN
2,Trimestre,Produit intérieur brut (PIB),Importations,Dépense de consommation des ménages,Dépense de consommation des APU1,Formation brute de capital fixe,Exportations
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-T3,0.362294,-0.684306,0.515766,0.513113,-0.84265,-0.470732


On remarque que les données sont ordonnées des plus récentes aux plus anciennes, on souhaite inverser cela pour faciliter le travail avec les autres tables. 

In [51]:
ev_pib = ev_pib.iloc[::-1].reset_index(drop=True)

In [52]:
# Vérification
ev_pib.head()

,trimestre,pib,importations,conso_menages,conso_apu,fbcf,exportations
0,"Source : Insee, comptes nationaux trimestriels...",NaN,NaN,NaN,NaN,NaN,NaN
1,Champ : France.,NaN,NaN,NaN,NaN,NaN,NaN
2,"Lecture : au 3e trimestre 2024, le produit int...",NaN,NaN,NaN,NaN,NaN,NaN
3,Note : données révisées ; les volumes sont mes...,NaN,NaN,NaN,NaN,NaN,NaN
4,1. Administrations publiques.,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# Trouver l'index de la valeur '2000-T1' dans la colonne 'trimestre'
index_2000_t1 = ev_pib[ev_pib['trimestre'] == '2000-T1'].index

# Afficher l'index
print(index_2000_t1)


Index([209], dtype='int64')


In [54]:
import pandas as pd

# On supprime les premières lignes inutiles pour que les données débutent au premier trimestre de l'année 2000
ev_pib_clean = ev_pib.iloc[209:].reset_index(drop=True)

# Afficher les premieres lignes des données nettoyées
ev_pib_clean.head()

,trimestre,pib,importations,conso_menages,conso_apu,fbcf,exportations
0,2000-T1,1.011252,4.716928,1.256031,0.606497,2.244349,2.839274
1,2000-T2,1.06425,4.20051,0.772804,0.464126,0.921508,4.963057
2,2000-T3,0.780148,3.529602,0.444674,0.34475,2.53796,1.41363
3,2000-T4,0.563163,2.2326,0.132284,0.155773,0.073162,2.984615
4,2001-T1,0.57489,-1.295035,1.139891,0.125288,0.672418,0.408129


On souhaite créer une nouvelle variable qui regroupe l'évolution de la consommation des ménages et des APU.

In [55]:
ev_pib_clean['conso'] = ev_pib_clean['conso_menages'] + ev_pib_clean['conso_apu']

In [56]:
# Afficher les premieres lignes des données 
ev_pib_clean.head()

,trimestre,pib,importations,conso_menages,conso_apu,fbcf,exportations,conso
0,2000-T1,1.011252,4.716928,1.256031,0.606497,2.244349,2.839274,1.862528
1,2000-T2,1.06425,4.20051,0.772804,0.464126,0.921508,4.963057,1.23693
2,2000-T3,0.780148,3.529602,0.444674,0.34475,2.53796,1.41363,0.789424
3,2000-T4,0.563163,2.2326,0.132284,0.155773,0.073162,2.984615,0.288057
4,2001-T1,0.57489,-1.295035,1.139891,0.125288,0.672418,0.408129,1.265179


In [57]:
ipc.head()

,Évolution de l’indice des prix à la consommation,Unnamed: 1
0,NaN,NaN
1,en %,NaN
2,Année,Taux d'inflation
3,2023,4.9
4,2022,5.2


In [58]:
# Renommer les colonnes à utiliser
ipc.rename(columns={'Évolution de l’indice des prix à la consommation': 'année'}, inplace=True)
ipc.rename(columns={'Unnamed: 1': 'ipc_année'}, inplace=True) 

In [59]:
print(ipc.columns)

Index(['année', 'ipc_année'], dtype='object')


In [60]:
ipc = ipc.iloc[::-1].reset_index(drop=True)

In [61]:
ipc.head()

,année,ipc_année
0,"Source : Insee, indices des prix à la consomma...",NaN
1,"Champ : France hors Mayotte, ensemble des ména...",NaN
2,"Lecture : en moyenne, sur l’année 2023, les pr...",NaN
3,1991,3.2
4,1992,2.4


In [62]:
# Trouver l'index de la valeur '2000' dans la colonne 'année'
index_2000 = ipc[ipc['année'] == '2000'].index

# Afficher l'index
print(index_2000)


Index([], dtype='int64')


In [63]:
# Convertir la colonne 'year' en datetime
ipc['année'] = pd.to_datetime(ipc['année'], format='%Y', errors='coerce')

# Filtrer les données à partir de l'année 2000
ipc = ipc[ipc['année'] >= '2000-01-01']

# Afficher les premières lignes des données filtrées
print(ipc)

        année ipc_année
12 2000-01-01       1.7
13 2001-01-01       1.6
14 2002-01-01       1.9
15 2003-01-01       2.1
16 2004-01-01       2.1
17 2005-01-01       1.7
18 2006-01-01       1.7
19 2007-01-01       1.5
20 2008-01-01       2.8
21 2009-01-01       0.1
22 2010-01-01       1.5
23 2011-01-01       2.1
24 2012-01-01         2
25 2013-01-01       0.9
26 2014-01-01       0.5
27 2015-01-01         0
28 2016-01-01       0.2
29 2017-01-01         1
30 2018-01-01       1.8
31 2019-01-01       1.1
32 2020-01-01       0.5
33 2021-01-01       1.6
34 2022-01-01       5.2
35 2023-01-01       4.9


Téléchargement des données sur le CAC 40

In [3]:
pip install yfinance


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.2/948.2 kB 41.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 34.4 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.8-cp312-cp312-linux_x86_64.whl size=301616 sha256=5dd6854d8c46808bff177debe519cf52653062ded718fc0b07db45d970ccca72
  Stored in directory: /home/onyxia/.cache/pip/wheels/8f/65/34/456800445efeafb05164fe95285c70e81ba1d96bae30f43917
Successfully built peewee
Note: you may need to restart the kernel to use updated packages.


In [4]:
import yfinance as yf
import pandas as pd

# Télécharger les données trimestrielles du CAC 40
ticker = "^FCHI"  # Symbole Yahoo Finance pour le CAC 40
data = yf.download(ticker, start="2000-01-01", interval="3mo")

# Afficher les données
print(data)

# Sauvegarder en CSV si nécessaire
data.to_csv("cac40_quarterly.csv")


[*********************100%***********************]  1 of 1 completed

Price         Adj Close        Close         High          Low         Open  \
Ticker            ^FCHI        ^FCHI        ^FCHI        ^FCHI        ^FCHI   
Date                                                                          
1999-12-01  6190.959961  6190.959961  6396.970215  5388.850098  6024.379883   
2000-03-01  6426.259766  6426.259766  6591.439941  5752.439941  6272.129883   
2000-06-01  6625.419922  6625.419922  6780.660156  6354.930176  6428.459961   
2000-09-01  5928.080078  5928.080078  6944.770020  5828.140137  6648.640137   
2000-12-01  5367.479980  5367.479980  6126.410156  5302.790039  5942.359863   
...                 ...          ...          ...          ...          ...   
2023-12-01  7927.430176  7927.430176  7977.680176  7281.100098  7336.549805   
2024-03-01  7992.870117  7992.870117  8259.190430  7899.359863  7950.660156   
2024-06-01  7630.950195  7630.950195  8072.310059  7029.910156  8069.620117   
2024-09-01  7235.109863  7235.109863  7804.640137  7